In [83]:
from parse_data import get_files, parse_files
from algorithm import build_dataframe, group_names
import pandas as pd
import pyperclip
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
calendars = get_files()
people = parse_files(calendars)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]

def build_dataframe(elements: list):
    frames = []
    for element in elements:
        frames.append(pd.DataFrame(element).dropna())

    df = pd.concat(frames)
    df.reset_index(inplace=True, drop=True)
    return df

def group_names(df: pd.DataFrame):
    lectures = []
    for Day in days:
        i = df.loc[df.Day == Day]
        lectures.append(
            i.groupby(["Day", "Start", "End", "Unit"], as_index=False).agg(
                {"Names": " ".join}
            )
        )
    lectures = pd.concat(lectures).reset_index(drop=True)

    del df["Names"]
    df = df.drop_duplicates(["Day", "Start", "End", "Group", "Building"])

    df.Day = pd.Categorical(df.Day, categories=days, ordered=True)
    df = df.sort_values("Day").reset_index(drop=True)

    df = df.merge(lectures)
    return df

In [92]:
def make_lalign_formatter(df, cols=None):
    """
    Construct formatter dict to left-align columns.

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The DataFrame to format
    cols : None or iterable of strings, optional
        The columns of df to left-align. The default, cols=None, will
        left-align all the columns of dtype object

    Returns
    -------
    dict
        Formatter dictionary

    """
    if cols is None:
       cols = df.columns[df.dtypes == 'object'] 

    return {col: f'{{:<{df[col].str.len().max()}s}}'.format for col in cols}


In [108]:

df = build_dataframe(people)
df = group_names(df)
df = df.loc[df.Type != "OL-Drop-In"]
x = []
for day in days:
    day_lects = df[df.Day == day]
    x.append("{1} {0:^20} {1}".format(day, "=================================="))
    x.append(df.to_string(formatters=make_lalign_formatter(day_lects, cols=["Room"]), index=False, justify="left"))



pyperclip.copy(x[1])



ValueError: Invalid format specifier